# VacationPy
---

In [24]:
# load dependencies
import hvplot.pandas
import pandas as pd
import requests
import os

# Import API key
from api_keys import geoapify_key

In [25]:
# read in the data generated by `WeatherPy.ipynb`
file_path = os.path.join('output_data', 'city_weather.csv')

city_data_df = pd.read_csv(file_path)[['City', 'Latitude', 'Longitude', 'Temperature (Max)', 'Humidity', 'Cloudiness', 'Wind Speed']]
city_data_df.head()

,City,Latitude,Longitude,Temperature (Max),Humidity,Cloudiness,Wind Speed
0,Palmer,61.599570,-149.111090,276.10,84,100,1.54
1,Tuktoyaktuk,69.444019,-133.031904,250.15,77,40,6.17
2,Ushuaia,-54.806116,-68.318497,278.98,70,75,6.69
3,Thule,77.468636,-69.222275,255.25,76,1,2.14
4,Zalantun City,47.746019,121.970532,284.04,19,4,4.44


In [30]:
# configure a map to show all cities in the dataframe with the size of the markers indicating humidity
map_plot = city_data_df.hvplot.points(
    'Longitude',
    'Latitude',
    geo = True,
    tiles = 'OSM',
    frame_width = 700,
    frame_height = 500,
    size = 'Humidity',
    scale = 0.8,
    color = 'City'
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity)